In [36]:
import requests
import os
from PIL import Image
import math

# Downloads a tile for the given x and y coordinates and zoom level. 
# It is pretty slow as it downloads one tile at a time
def download_tile(x, y, z): 
    url = "https://khms.google.com/kh/v=908?x=" + str(x) + "&y=" + str(y) + "&z=" + str(z)
    print(url)
    imagefile = requests.get(url)
    try:
        os.mkdir("./tiles/")
    except:
        q = 0 # Do nothing if the folder exists
    filename = './tiles/tile_' + str(x) + '_' + str(y) + '_' + str(z) + '.jpeg'
    open(filename, 'wb').write(imagefile.content)
    print("Downloading:", filename)

# Unused, but might add it later
def dl_all_tiles_from_zoom(zoom, stitch=False): 
    xtile = 0
    ytile = 0
    max_tile = 2**zoom-1
    while xtile <= max_tile:
        while ytile <= max_tile:
            download_tile(xtile, ytile, zoom)
            ytile += 1
        ytile = 0
        xtile += 1
    print("Finished downloading files.")


# Stitches tiles into one large image
def stitch_tiles(x, y, size, z):
    width = size * 256
    height = size * 256
    max_x = x + size - 1
    max_y = y + size - 1
    x2 = x
    y2 = y
    progress = 0
    total = size*size
    output = Image.new(mode = "RGB", size = (width, height))
    while x2 <= max_x:
        while y2 <= max_y:
            img = Image.open('tiles/tile_' + str(x2) + '_' + str(y2) + '_' + str(z) + '.jpeg')
            position_x = (x2 - x) * 256
            position_y = (y2 - y) * 256
            output.paste(img, (position_x, position_y))
            y2 += 1
            progress += 1
            print("Stitching:", str(progress) + "/" + str(total))
        y2 = y
        x2 += 1
    print("Writing stitched image to file...")
    output.save('stitched.jpg')
    print("Done stitching images.")

# Downloads a square of tiles, might add other shapes later
def dl_square(x, y, z, size): 
    max_x = x + size - 1
    max_y = y + size - 1
    y2 = y
    print(f'x: {x}\nmax_x:{max_x}\ny:{y}\nmax_y: {max_y}' )
    while x <= max_x:
        while y <= max_y:
            download_tile(x, y, z)
            y += 1
        y = y2
        x += 1

def latlong_to_xy(lat, lon, zoom): # Converts latitude and longitude to tile coordinates
    lonRad = math.radians(lon)
    latRad = math.radians(lat)
    columnIndex = lonRad
    rowIndex = math.log(math.tan(latRad) + (1.0 / math.cos(latRad)))
    columnNormalized = (1 + (columnIndex / math.pi)) / 2
    rowNormalized = (1 - (rowIndex / math.pi)) / 2
    tilesPerRow = 2 ** zoom
    column = round(columnNormalized * (tilesPerRow - 1))
    row = round(rowNormalized * (tilesPerRow - 1))
    return [column, row]

In [50]:
# lat = float(input("Enter latitude:"))
# longi = float(input("Enter longitude:"))
# zoom = int(input("Enter zoom level:"))
# size_deg = float(input("Enter size in longitute degrees:"))

lat = 45.2245949
longi = 5.6858155
zoom = 13
size_deg = 0.3 
size = int(size_deg * 2 ** zoom / 360)
coords = latlong_to_xy(lat, longi, zoom)
dl_square(coords[0], coords[1], zoom, size)
stitch_tiles(coords[0], coords[1], size, zoom)

x: 4225
max_x:4230
y:2939
max_y: 2944
https://khms.google.com/kh/v=908?x=4225&y=2939&z=13
Downloading: ./tiles/tile_4225_2939_13.jpeg
https://khms.google.com/kh/v=908?x=4225&y=2940&z=13
Downloading: ./tiles/tile_4225_2940_13.jpeg
https://khms.google.com/kh/v=908?x=4225&y=2941&z=13
Downloading: ./tiles/tile_4225_2941_13.jpeg
https://khms.google.com/kh/v=908?x=4225&y=2942&z=13
Downloading: ./tiles/tile_4225_2942_13.jpeg
https://khms.google.com/kh/v=908?x=4225&y=2943&z=13
Downloading: ./tiles/tile_4225_2943_13.jpeg
https://khms.google.com/kh/v=908?x=4225&y=2944&z=13
Downloading: ./tiles/tile_4225_2944_13.jpeg
https://khms.google.com/kh/v=908?x=4226&y=2939&z=13
Downloading: ./tiles/tile_4226_2939_13.jpeg
https://khms.google.com/kh/v=908?x=4226&y=2940&z=13
Downloading: ./tiles/tile_4226_2940_13.jpeg
https://khms.google.com/kh/v=908?x=4226&y=2941&z=13
Downloading: ./tiles/tile_4226_2941_13.jpeg
https://khms.google.com/kh/v=908?x=4226&y=2942&z=13
Downloading: ./tiles/tile_4226_2942_13.jpeg
ht

In [51]:
#meters_per_pixel = 156543.03392 * Math.cos(latLng.lat() * Math.PI / 180) / Math.pow(2, zoom)
meters_per_pixel = 156543.03392 * math.cos(lat * math.pi / 180) / (2 ** zoom)
meters_per_pixel

13.459214496431462